It's not possible to reference data or vairables across different languages in a notebook.  But you can you temporary objects to get around this limitation, creatively.

This notebook also shows how to connect to a WASB account 

**Make sure you complete Lab0005 first.**

In [12]:
from pyspark.sql import SparkSession

# Azure storage access info
# we can use my read-only data lake
# you should've been a connection to this in Lab005 already
blob_account_name = 'davewdemoblobs' # replace with your blob name
blob_container_name = 'nyctaxi-staging' # replace with your container name
blob_relative_path = 'reference-data/payment_type_lookup.csv' # replace with your relative folder path
linked_service_name = 'davesdemoblobs' # replace with your linked service name

# this should work but is not.  Not sure why
# blob_sas_token = mssparkutils.credentials.getConnectionStringOrCreds(linked_service_name)
# for now, we can do this manually, but definitely not a best practice
# get the token from the git repo, this can't be committed
blob_sas_token = 'sv=...'


# Allow SPARK to access from Blob remotely

wasb_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)

spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name), blob_sas_token)
print('Remote blob path: ' + wasb_path)

StatementMeta(SparkPool01, 6, 12, Finished, Available)

Remote blob path: wasbs://nyctaxi-staging@davewdemoblobs.blob.core.windows.net/reference-data/payment_type_lookup.csv

In [15]:
dfPayTypes = spark.read.option("header", "true") \
            .option("delimiter","|") \
            .csv(wasb_path)
            #.schema(schema) \

StatementMeta(SparkPool01, 6, 15, Finished, Available)

In [16]:
dfPayTypes.show()

StatementMeta(SparkPool01, 6, 16, Finished, Available)

+------------+------------+-----------+
|payment_type|abbreviation|description|
+------------+------------+-----------+
|           1|        null|Credit card|
|           2|        null|       Cash|
|           3|        null|  No charge|
|           4|        null|    Dispute|
|           5|        null|    Unknown|
|           6|        null|Voided trip|
|           7|         CAS|       Cash|
|           8|        CASH|       Cash|
|           9|         CRD|Credit card|
|          10|         CRE|Credit card|
|          11|      CREDIT|Credit card|
|          12|         CSH|       Cash|
|          13|         Cas|       Cash|
|          14|        Cash|       Cash|
|          15|         Cre|Credit card|
|          16|      Credit|Credit card|
|          17|         DIS|    Dispute|
|          18|         Dis|    Dispute|
|          19|     Dispute|    Dispute|
|          20|         NA |    Unknown|
+------------+------------+-----------+
only showing top 20 rows

In [17]:
# register df as a temp table
# I like to keep the names the same to avoid confusion
dfPayTypes.registerTempTable("dfPayTypes")

StatementMeta(SparkPool01, 6, 17, Finished, Available)

In [18]:
%%sql
--now let's manipulate this table from SparkSQL
select * from dfPayTypes

StatementMeta(SparkPool01, 6, 18, Finished, Available)

<Spark SQL result set with 24 rows and 3 fields>

In [22]:
%%spark
//that is the magic for scala.  
//TODO:  need to figure out how to do this.  I can do it with an externally managed table,
//this is proving difficult

//let's look at the table from Scala
//sqlContext.sql("select * from dfPayTypes")
//val dfPayTypes = spark.read.synapsesql("sqlpool.dbo.dfPayTypes")

StatementMeta(SparkPool01, 6, 23, Finished, Available)

Error: StructuredStream-spark package version: 2.4.5-1.3.1

In [1]:
%%spark
//this is the code for scala to read a table from Serverless SQL Pool
val scalaDataFrame = spark.read.sqlanalytics("mySQLPoolDatabase.dbo.mySQLPoolTable")
scalaDataFrame.createOrReplaceTempView( "mydataframetable" )


In [2]:
%%sql
SELECT * FROM mydataframetable